In [1]:
import requests
import json
import time
import pandas as pd

file = open("GeoapiKey.txt","r")
GeoapiKey = file.read()
file.close()

file = open("google_apiKey.txt","r")
apiKey = file.read()
file.close()

In [2]:
class GoogleGeocode(object):
    def __init__(self, GeoapiKey):
        super(GoogleGeocode, self).__init__()
        self.apiKey = GeoapiKey
        
    def get_coordinates(self, address):
        endpoint_url = 'https://maps.googleapis.com/maps/api/geocode/json'
        params = {
            'address': address,
            'key': self.apiKey
        }
        res = requests.get(endpoint_url, params = params)
        georesults =  json.loads(res.content)
        lat = georesults['results'][0]['geometry']['location']['lat']
        lng = georesults['results'][0]['geometry']['location']['lng']
        cordinates = str(lat) + ',' + str(lng)
        return(cordinates)
        

In [3]:
class GooglePlaces(object):
    def __init__(self, apiKey):
        super(GooglePlaces, self).__init__()
        self.apiKey = apiKey
 
    def search_places_by_coordinate(self, location, types):
        endpoint_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
        places = []
        params = {
            'location': location,
            'types': types,
            'rankby' : 'distance',
            'key': self.apiKey
        }
        res = requests.get(endpoint_url, params = params)
        results =  json.loads(res.content)
        places.extend(results['results'])
        time.sleep(2)
        
        while "next_page_token" in results:
            params['pagetoken'] = results['next_page_token'],
            res = requests.get(endpoint_url, params = params)
            results = json.loads(res.content)
            places.extend(results['results'])
            time.sleep(2)
        
        return(places)

    
    def get_place_details(self, place_id, fields):
        endpoint_url = "https://maps.googleapis.com/maps/api/place/details/json"
        params = {
            'placeid': place_id,
            'fields': ",".join(fields),
            'key': self.apiKey
        }
        res = requests.get(endpoint_url, params = params)
        place_details =  json.loads(res.content)
        return(place_details)

In [4]:
geoapi = GoogleGeocode(GeoapiKey)

In [5]:
cordinates = geoapi.get_coordinates('Missoula')

In [6]:
cordinates

'46.8721284,-113.9940314'

In [7]:
api = GooglePlaces(apiKey)

In [8]:
places = api.search_places_by_coordinate(cordinates, types = 'restaurant')

In [9]:
places[0]

{'business_status': 'OPERATIONAL',
 'geometry': {'location': {'lat': 46.8721284, 'lng': -113.9940314},
  'viewport': {'northeast': {'lat': 46.8734919802915,
    'lng': -113.9927289197085},
   'southwest': {'lat': 46.87079401970851, 'lng': -113.9954268802915}}},
 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/restaurant-71.png',
 'icon_background_color': '#FF9E67',
 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/restaurant_pinlet',
 'name': 'Zone 4061',
 'opening_hours': {'open_now': True},
 'place_id': 'ChIJoy1gRTzNXVMRymPfV2SZjaQ',
 'plus_code': {'compound_code': 'V2C4+V9 Missoula, MT, USA',
  'global_code': '85R8V2C4+V9'},
 'reference': 'ChIJoy1gRTzNXVMRymPfV2SZjaQ',
 'scope': 'GOOGLE',
 'types': ['restaurant', 'point_of_interest', 'food', 'establishment'],
 'vicinity': 'Missoula'}

Big limitation can only resturn 60 results. They used to have a radar search that would return 200 results. It was taken down. Google has it posted all over the web that workarounds are agianst their service terms to prevent web scraping. I decided not to take this risk.

In [10]:
len(places)

60

In [11]:
fields = ['name', 'user_ratings_total', 'formatted_address', 'opening_hours', 'permanently_closed', 'price_level', 'international_phone_number', 'website', 'rating', 'review']

In [12]:
restaurants_list=[]

for place in places:
    details = api.get_place_details(place['place_id'], fields)
    restaurants_list.append(pd.json_normalize(details))

restaurants = pd.concat(restaurants_list)

In [13]:
len(restaurants)

60

In [14]:
restaurants.drop(['html_attributions',
                  'status',
                  'result.opening_hours.open_now',
                  'result.opening_hours.periods']
                 , axis=1, inplace=True)

In [15]:
restaurants.rename(columns = {'result.formatted_address':'Address', 
                              'result.international_phone_number':'Phone_Number', 
                              'result.name':'Name',
                              'result.opening_hours.weekday_text': 'Hours_Open',
                              'result.price_level' : 'Price_level',
                              'result.reviews' : 'reviews',
                              'result.website' : 'website',
                              'result.rating': 'Rating',
                              'result.user_ratings_total': '#_of_Ratings'}, inplace = True)

In [16]:
restaurants = restaurants.set_index('Name')

In [17]:
restaurants = restaurants.sort_values('Rating',  ascending=False)

In [18]:
restaurants.head()

,Address,Phone_Number,Hours_Open,Rating,reviews,#_of_Ratings,website,Price_level,result.permanently_closed
Name,,,,,,,,,
Clove Cart Pizza,"Mobile Pizza Oven, Missoula, MT 59802, USA",+1 617-571-6894,"[Monday: Closed, Tuesday: Closed, Wednesday: 1...",5.0,"[{'author_name': 'Kaylee Swope', 'author_url':...",9.0,http://www.clovecart.com/,NaN,NaN
Gary's Local $6 Burgers,"235 E Front St, Missoula, MT 59802, USA",NaN,"[Monday: 12:00 – 2:30 PM, 5:00 – 11:00 PM, Tue...",4.9,"[{'author_name': 'Luke Meissner', 'author_url'...",47.0,https://order.toasttab.com/online/garys-local,2.0,NaN
Tea & Crepe Missoula,"140 N Higgins Ave #103, Missoula, MT 59802, USA",+1 406-493-0271,"[Monday: 11:30 AM – 9:30 PM, Tuesday: 11:30 AM...",4.8,"[{'author_name': 'Brooke Blackler', 'author_ur...",115.0,http://teacrepemissoula.com/,NaN,NaN
Basal,"114 N Higgins Ave, Missoula, MT 59802, USA",+1 406-540-4098,"[Monday: 8:00 AM – 8:00 PM, Tuesday: 8:00 AM –...",4.7,"[{'author_name': 'L G', 'author_url': 'https:/...",209.0,https://www.basalhospitality.com/,2.0,NaN
Wally and Buck,"319 E Front St, Missoula, MT 59802, USA",+1 406-549-8157,"[Monday: 11:00 AM – 9:00 PM, Tuesday: 11:00 AM...",4.7,"[{'author_name': 'Sabrina Charlson', 'author_u...",395.0,http://wallyandbuck.com/,NaN,NaN


In [19]:
review_dictionary = {}
for key, value in restaurants.iterrows():
    rest_review = value['reviews']
    if type(rest_review) == float:
        continue
    else:
        review_dictionary[key] = pd.json_normalize(rest_review)

In [20]:
review_dictionary['Basal']

,author_name,author_url,language,original_language,profile_photo_url,rating,relative_time_description,text,time,translated
0,L G,https://www.google.com/maps/contrib/1149292388...,en,en,https://lh3.googleusercontent.com/a/ACg8ocJekW...,5,5 months ago,"AMAZING! fresh, delicious, filling and nutriti...",1692595700,False
1,Nick Rohleder,https://www.google.com/maps/contrib/1158258696...,en,en,https://lh3.googleusercontent.com/a/ACg8ocIhOg...,5,5 months ago,This restaurant is amazing. It's a little expe...,1694724118,False
2,Erin Burke,https://www.google.com/maps/contrib/1010320170...,en,en,https://lh3.googleusercontent.com/a-/ALV-UjVno...,5,4 months ago,Fresh and delicious! Been here three times so ...,1696956268,False
3,Hanna Reese,https://www.google.com/maps/contrib/1078811431...,en,en,https://lh3.googleusercontent.com/a-/ALV-UjVYn...,5,a week ago,I normally never write reviews--having been in...,1707259211,False
4,Green Deathray,https://www.google.com/maps/contrib/1097840934...,en,en,https://lh3.googleusercontent.com/a/ACg8ocJyQh...,5,6 months ago,Lovely downtown restaurant with salads and hea...,1692556100,False


In [21]:
# for key in review_dictionary:
#     review_dictionary[key].to_csv('Google_Reviews/' + str(key) + '_Google_reviews.csv')

In [22]:
# restaurants.to_csv('Google_Missoula_Restaurants.csv')